<a href="https://colab.research.google.com/github/Fakhar27/text_to_speech_SunoBark/blob/main/bark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***SIMPLIFIED BARK IMPLEMENTATION BY FAKHAR***

In [1]:
!pip install --upgrade --quiet pip
!pip install --quiet transformers torch scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 156.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 176.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.1 MB/s eta 0:00:00


In [2]:
import torch
from transformers import BarkModel, AutoProcessor
from IPython.display import Audio
import scipy.io.wavfile

In [9]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = BarkModel.from_pretrained("suno/bark").to(device)
processor = AutoProcessor.from_pretrained("suno/bark")

config.json:   0%|          | 0.00/8.81k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.49G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/4.91k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/353 [00:00<?, ?B/s]

speaker_embeddings_path.json:   0%|          | 0.00/61.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [10]:
def generate_speech(text, voice_preset=None):
    inputs = processor(text, voice_preset=voice_preset)
    inputs = {k: v.to(device) if hasattr(v, 'to') else v for k, v in inputs.items()}
    speech_output = model.generate(**inputs)
    return speech_output

In [11]:
text = "Suddenly, a bright flash illuminated the night sky, and a deafening thunderclap shook the ground, startling the adventurers and sending their meager fire pit flying."
speech_output = generate_speech(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [12]:
sampling_rate = model.generation_config.sample_rate
Audio(speech_output[0].cpu().numpy(), rate=sampling_rate)

In [ ]:
scipy.io.wavfile.write(
    "output.wav",
    rate=sampling_rate,
    data=speech_output[0].cpu().numpy()
)

# ***ABOVE PART IS SIMPLE USAGE OF BARK NOW WE MOVE ON TOO COMPLEX AND OPTIMIZED VERSION***

In [ ]:
!pip install --quiet --upgrade pip
!pip install --quiet transformers torch scipy
!pip install --quiet accelerate
!pip install --quiet optimum
!pip install -U flash-attn --no-build-isolation

In [5]:
import torch
from transformers import BarkModel, AutoProcessor
from IPython.display import Audio
import scipy.io.wavfile

In [6]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model = BarkModel.from_pretrained(
    "suno/bark",
    torch_dtype=torch.float16,
).to(device)

Using device: cuda:0


pytorch_model.bin:   0%|          | 0.00/4.49G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/4.91k [00:00<?, ?B/s]

In [7]:
try:
    model = model.to_bettertransformer()
    print("BetterTransformer optimization enabled")
except Exception as e:
    print(f"BetterTransformer optimization failed: {e}")

processor = AutoProcessor.from_pretrained("suno/bark")

def generate_speech(text, voice_preset=None):
    inputs = processor(text, voice_preset=voice_preset)
    inputs = {k: v.to(device) if hasattr(v, 'to') else v for k, v in inputs.items()}
    with torch.inference_mode():
        speech_output = model.generate(**inputs)
    return speech_output

The class `optimum.bettertransformers.transformation.BetterTransformer` is deprecated and will be removed in a future release.
The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


BetterTransformer optimization enabled


In [10]:
test_text = "As they ascended the steep mountain face, a sudden blizzard engulfed them, reducing visibility to mere inches"

print("\nStarting generation...")
speech_output = generate_speech(test_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.



Starting generation...


In [11]:
sampling_rate = model.generation_config.sample_rate
Audio(speech_output[0].cpu().numpy(), rate=sampling_rate)

In [ ]:
scipy.io.wavfile.write(
    "bark_output.wav",
    rate=sampling_rate,
    data=speech_output[0].cpu().numpy()
)

## ***NOW WORKING WITH TRANSFORMERS FLASH ATTENTION 2 FOR FURTHER ENHANCEMENTS (WILL ONLY WORK WITH CUDA SUPPORTED GPUS LIKE A100,H100)***

In [ ]:
!pip install --quiet --upgrade pip
!pip install --quiet transformers torch scipy
!pip install --quiet accelerate optimum
!pip install -U flash-attn --no-build-isolation

In [ ]:
import torch
from transformers import BarkModel, AutoProcessor
from IPython.display import Audio
import scipy.io.wavfile
import platform
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def check_flash_attention_compatibility():
    """
    Check if the current GPU supports Flash Attention 2
    """
    if not torch.cuda.is_available():
        return False

    gpu_name = torch.cuda.get_device_name()
    compute_capability = torch.cuda.get_device_capability()

    is_compatible = (
        'A100' in gpu_name or
        'H100' in gpu_name or
        'RTX 30' in gpu_name or
        'RTX 40' in gpu_name or
        compute_capability[0] >= 8
    )

    print(f"GPU: {gpu_name}")
    print(f"Compute Capability: {compute_capability}")
    print(f"Flash Attention 2 Compatible: {is_compatible}")

    return is_compatible

In [ ]:
def load_optimized_model():
    """
    Load model with best available optimizations
    """
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")
    use_flash = check_flash_attention_compatibility()

    try:
        if use_flash:
            print("Loading model with Flash Attention 2...")
            model = BarkModel.from_pretrained(
                "suno/bark",
                torch_dtype=torch.float16,
                attn_implementation="flash_attention_2"
            ).to(device)
            print("Successfully loaded with Flash Attention 2")
        else:
            print("Loading model with BetterTransformer...")
            model = BarkModel.from_pretrained(
                "suno/bark",
                torch_dtype=torch.float16
            ).to(device)
            model = model.to_bettertransformer()
            print("Successfully loaded with BetterTransformer")
        model.enable_cpu_offload()
        print("CPU offloading enabled")

    except Exception as e:
        print(f"Optimization failed: {e}")
        print("Falling back to basic model...")
        model = BarkModel.from_pretrained(
            "suno/bark",
            torch_dtype=torch.float16
        ).to(device)

    return model

def generate_optimized_speech(text, model, processor, voice_preset=None):
    """
    Generate speech with memory tracking and optimization
    """
    device = next(model.parameters()).device
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print(f"\nInitial GPU Memory: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
    inputs = processor(text, voice_preset=voice_preset)
    inputs = {k: v.to(device) if hasattr(v, 'to') else v for k, v in inputs.items()}
    with torch.inference_mode(), torch.cuda.amp.autocast(dtype=torch.float16):
        speech_output = model.generate(**inputs)
    if torch.cuda.is_available():
        print(f"Final GPU Memory: {torch.cuda.memory_allocated()/1024**2:.2f} MB")

    return speech_output

In [ ]:
print("Initializing Bark with optimizations...")
model = load_optimized_model()
processor = AutoProcessor.from_pretrained("suno/bark")

In [ ]:
test_text = """
In the depths of the ancient forest, where shadows danced between towering trees and mystical fog swirled around gnarled roots,
a group of adventurers cautiously made their way forward. The air was thick with the scent of moss and magic,
and every step echoed with the weight of untold possibilities.
"""

print("\nStarting generation...")
speech_output = generate_optimized_speech(test_text, model, processor)

sampling_rate = model.generation_config.sample_rate
Audio(speech_output[0].cpu().numpy(), rate=sampling_rate)

scipy.io.wavfile.write(
    "optimized_bark_output.wav",
    rate=sampling_rate,
    data=speech_output[0].cpu().numpy()
)

## ***WORKING WITH LONGER PROMPTS***

In [5]:
!pip install --quiet --upgrade pip
!pip install --quiet transformers torch scipy nltk optimum

In [6]:
import torch
from transformers import BarkModel, AutoProcessor
from IPython.display import Audio
import scipy.io.wavfile
import numpy as np
import nltk

print("Downloading required NLTK data...")
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
class LongFormBark:
    def __init__(self):
        self.device = "cuda:0" if torch.cuda.is_available() else "cpu"
        print(f"Using device: {self.device}")
        print("Loading model...")
        self.model = BarkModel.from_pretrained(
            "suno/bark",
            torch_dtype=torch.float16,
        ).to(self.device)
        try:
            if 'optimum' in globals():
                self.model = self.model.to_bettertransformer()
                print("BetterTransformer optimization enabled")
        except Exception as e:
            print(f"BetterTransformer not available: {e}")
            print("Continuing with base model")

        self.processor = AutoProcessor.from_pretrained("suno/bark")
        self.sample_rate = self.model.generation_config.sample_rate

    def split_into_sentences(self, text):
        """Split text into sentences using basic punctuation if NLTK fails"""
        try:
            return nltk.sent_tokenize(text)
        except Exception as e:
            print(f"NLTK tokenization failed: {e}")
            print("Falling back to basic sentence splitting")
            sentences = []
            current = []
            for piece in text.replace('\n', ' ').split('. '):
                current.append(piece)
                if len(' '.join(current)) > 100:
                    sentences.append('. '.join(current) + '.')
                    current = []
            if current:
                sentences.append('. '.join(current) + '.')
            return sentences

    def generate_speech_chunk(self, text_chunk, voice_preset=None):
        """Generate speech for a single chunk of text"""
        inputs = self.processor(text_chunk, voice_preset=voice_preset)
        inputs = {k: v.to(self.device) if hasattr(v, 'to') else v for k, v in inputs.items()}

        with torch.inference_mode():
            speech_output = self.model.generate(**inputs)

        return speech_output[0].cpu().numpy()

    def generate_long_form(self, text, voice_preset=None, silence_duration=0.25):
        """Generate speech for long text by splitting into chunks"""
        sentences = self.split_into_sentences(text)
        print(f"Text split into {len(sentences)} chunks")
        silence = np.zeros(int(silence_duration * self.sample_rate))
        pieces = []
        for i, sentence in enumerate(sentences, 1):
            print(f"\nProcessing chunk {i}/{len(sentences)}")
            print(f"Text: {sentence[:50]}...")

            try:
                audio_chunk = self.generate_speech_chunk(sentence, voice_preset)
                pieces.extend([audio_chunk, silence.copy()])

                if torch.cuda.is_available():
                    torch.cuda.empty_cache()

            except Exception as e:
                print(f"Error processing chunk {i}: {e}")
                continue

        return np.concatenate(pieces)

    def save_audio(self, audio_array, filename="bark_long_output.wav"):
        """Save the generated audio to a file"""
        scipy.io.wavfile.write(
            filename,
            rate=self.sample_rate,
            data=audio_array
        )
        print(f"Audio saved to {filename}")

In [8]:
bark_generator = LongFormBark()

Using device: cuda:0
Loading model...


In [9]:
test_text = """
In the depths of the ancient forest, where shadows danced between towering trees and mystical fog swirled around gnarled roots,
a group of adventurers cautiously made their way forward. The air was thick with the scent of moss and magic,
and every step echoed with the weight of untold possibilities.
"""

print("\nGenerating long-form audio...")
audio_array = bark_generator.generate_long_form(
    test_text,
    voice_preset="v2/en_speaker_6",
    silence_duration=0.5
)


Generating long-form audio...
NLTK tokenization failed: 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

Falling back to basic sentence splitting
Text split into 3 chunks

Processing chunk 1/3
Text:  In the depths of the ancient forest, where shadow...


en_speaker_6_semantic_prompt.npy:   0%|          | 0.00/2.60k [00:00<?, ?B/s]

en_speaker_6_coarse_prompt.npy:   0%|          | 0.00/7.55k [00:00<?, ?B/s]

en_speaker_6_fine_prompt.npy:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Processing chunk 2/3
Text: The air was thick with the scent of moss and magic...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.



Processing chunk 3/3
Text: ....


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [10]:
Audio(audio_array, rate=bark_generator.sample_rate)